In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

##############################################################################

df = pd.read_csv('D:\data from MnM Samdal raw\Samdal_power_1s_no_outliers_after_window_30s_20perc_spd10_pow.csv', parse_dates=[1], index_col=1 )

del df['Unnamed: 0']
del df['date']
del df['time']
del df['maxs_pow']
del df['maxs_spd']
del df['mins_pow']
del df['mins_spd']
del df['differences_spd']
del df['differences_pow']


In [8]:
# divide into subsets 

data2 = df.ix[180000:350000, ['speed', 'power']]
data2 = data2.as_matrix().astype("float64", copy = False)

X = data2
X = StandardScaler().fit_transform(X)

In [9]:
%%time
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")


db = DBSCAN(eps=0.1, min_samples=60).fit(X) 

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# # Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = plt.get_cmap('Set2')(np.linspace(0, 1, len(unique_labels)))

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = 'k'

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col, markeredgecolor='k', markeredgewidth = 1, markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col, markeredgecolor='k', markeredgewidth = 1, markersize=6)

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.title('Estimated number of clusters: %d' % n_clusters_, fontsize = 26)
plt.xlabel('normalized wind speed', fontsize = 26)
plt.ylabel('normalized power', fontsize = 26)
plt.show()


C:\Anaconda3\lib\site-packages\matplotlib\lines.py:1106: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._markerfacecolor != fc:


Wall time: 4min 2s
